<a href="https://colab.research.google.com/github/acollin19/comp551/blob/Angele/A2/assignment2_group_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

## Process:

### Task 1: Data Processing  


1. IMDB Reviews
2. 20 news groups: a multi-class labelled textual dataset


### Task 2: Implement Logistic and Multiclass classifiers

1. Logistic Regression
2. Multiclass Regression
3. Comparisons using ROC curve


### Task 3: Running Experiments



1.    Most positive/negative Z-scores
2.   Implement:  
    * Binary classification on the IMDB Reviews

  *   Multi-class classification on the 20 news group dataset

3. Binary classification AUROC on IMDB data
  *   Logistic Regression 
  *   KNN

4. Multiclass classification accuracy

5. Compare Accuracy of Models (Plot)







In [18]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_svmlight_file
from scipy.sparse import csr_matrix

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Multiclass Regression and ROC curve comparisons
from sklearn import model_selection
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression as sk_LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from nltk import word_tokenize, download
from nltk.corpus import stopwords


In [19]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!ls aclImdb
!ls aclImdb/test
!ls aclImdb/train


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  14.9M      0  0:00:05  0:00:05 --:--:-- 16.8M
imdbEr.txt  imdb.vocab	README	test  train
labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt
labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [20]:
# IMDB dataset preprocession
IMDB_X_train, IMDB_y_train= load_svmlight_file('aclImdb/train/labeledBow.feat')
IMDB_X_test, IMDB_y_test = load_svmlight_file('aclImdb/test/labeledBow.feat') 

print(IMDB_X_train)

  (0, 0)	9.0
  (0, 1)	1.0
  (0, 2)	4.0
  (0, 3)	4.0
  (0, 4)	6.0
  (0, 5)	4.0
  (0, 6)	2.0
  (0, 7)	2.0
  (0, 8)	4.0
  (0, 10)	4.0
  (0, 12)	2.0
  (0, 26)	1.0
  (0, 27)	1.0
  (0, 28)	1.0
  (0, 29)	2.0
  (0, 32)	1.0
  (0, 41)	1.0
  (0, 45)	1.0
  (0, 47)	1.0
  (0, 50)	1.0
  (0, 54)	2.0
  (0, 57)	1.0
  (0, 59)	1.0
  (0, 63)	2.0
  (0, 64)	1.0
  :	:
  (24999, 420)	1.0
  (24999, 421)	1.0
  (24999, 426)	1.0
  (24999, 427)	1.0
  (24999, 583)	1.0
  (24999, 585)	1.0
  (24999, 642)	1.0
  (24999, 679)	2.0
  (24999, 680)	1.0
  (24999, 1065)	1.0
  (24999, 1093)	1.0
  (24999, 1224)	1.0
  (24999, 1407)	1.0
  (24999, 1773)	1.0
  (24999, 3947)	1.0
  (24999, 4292)	1.0
  (24999, 4569)	1.0
  (24999, 4949)	1.0
  (24999, 5072)	1.0
  (24999, 5792)	1.0
  (24999, 5947)	1.0
  (24999, 9702)	1.0
  (24999, 12190)	1.0
  (24999, 12803)	1.0
  (24999, 15612)	1.0


In [21]:
# 1st element in tuple represents the file its in, 2nd element represents the index of word it represents in vocab, float represents frequency
# want to remove the words that appear in less than 1% of all documents # only appear in 250 documents
# want to remove the words that appear in more than 50% of all documents # appear in 12500 documents
import scipy.sparse as sp
sign = sum(IMDB_X_train.sign()) # 1 if the word appears in the text 0 if not
good = (sign < 12500) - (sign < 250)
indices = sp.find(good)

IMDB_X_train = IMDB_X_train[:,indices[1]]
IMDB_X_train.todense()
print(IMDB_X_train)

  (0, 2)	1.0
  (0, 5)	1.0
  (0, 14)	1.0
  (0, 18)	1.0
  (0, 20)	1.0
  (0, 23)	1.0
  (0, 27)	2.0
  (0, 30)	1.0
  (0, 32)	1.0
  (0, 36)	2.0
  (0, 37)	1.0
  (0, 39)	1.0
  (0, 41)	2.0
  (0, 43)	1.0
  (0, 45)	1.0
  (0, 51)	1.0
  (0, 73)	1.0
  (0, 79)	1.0
  (0, 89)	1.0
  (0, 95)	1.0
  (0, 98)	1.0
  (0, 109)	1.0
  (0, 113)	1.0
  (0, 115)	1.0
  (0, 123)	1.0
  :	:
  (24999, 176)	1.0
  (24999, 178)	1.0
  (24999, 206)	1.0
  (24999, 228)	1.0
  (24999, 240)	1.0
  (24999, 272)	1.0
  (24999, 277)	1.0
  (24999, 285)	1.0
  (24999, 310)	1.0
  (24999, 317)	1.0
  (24999, 320)	1.0
  (24999, 364)	1.0
  (24999, 393)	1.0
  (24999, 394)	1.0
  (24999, 399)	1.0
  (24999, 400)	1.0
  (24999, 556)	1.0
  (24999, 558)	1.0
  (24999, 615)	1.0
  (24999, 652)	2.0
  (24999, 653)	1.0
  (24999, 1038)	1.0
  (24999, 1066)	1.0
  (24999, 1195)	1.0
  (24999, 1369)	1.0


## Z-Score Function

In [25]:
from sklearn.utils.extmath import sparse
# Z - Score
def zscore(X,y):

  N = X.shape[0]
  std = (IMDB_y_train- y.mean()) / y.std()
  z = np. abs(sp.csr_matrix.dot(sp.csr_matrix.transpose(X), std))

  return z


df = pd.DataFrame(IMDB_X_train.toarray())
df.insert(loc=0, column="A", value=IMDB_y_train)
IMDB_X_train = sp.csr_matrix(df.values)
IMDB_X_train.todense()
IMDB_arr = IMDB_X_train.toarray()

zscores = zscore(IMDB_X_train,IMDB_y_train)

zindex = np.argsort(zscores)[::-1][:300]

# get correct indexed feature set
new_IMDB_X_train = IMDB_X_train[:,zindex]
new_IMDB_X_test = IMDB_X_test[:,zindex]
# new_IMDB_y_train = IMDB_y_train[zindex]
# new_IMDB_y_test = IMDB_y_test[zindex]


# get correct indexed word list
vocab = pd.read_csv('aclImdb/imdb.vocab', names=['Words'])
vocab = vocab['Words'].values
word_list = vocab[indices[1]]
word_list = [word_list[i] for i in zindex]

# print(zscores.shape[0]) #1746

In [28]:
print(zindex)
# print(zscores)
print(new_IMDB_X_train.shape[0])


# IMDB_y_train[IMDB_y_train <= 4] = -1.0
# IMDB_y_train[IMDB_y_train >= 7] = 1.0
# print(IMDB_y_train)


[   0    1   21   52    3   58   30   15   28    8   12  215   14   27
   90    2   13   89   11   17   10   54   63   86   84   46  131   91
   35   33   18  108   72   36    5   69  343   70  402   66   56  200
  284  348   51  124    7  338   26  344   42  319  196  397  125  242
  165   61  303    9  272  149  154  478    6  104  117  204   40   62
   22  101   98  179  129  198   81   64  434  191  184  174   25  100
  367   48   57  382   19  113  559   24  412   65   38  114  134   99
  199  468   49   93  119  269  187   92   16  245   37   20  414  220
  484   96  171   79  162  169  315  254  285  235  597  464  243   31
    4  123  236  160  248  384  251  151  504   60  148  735  227  318
  257  573  221   53  132  788  346  106  831  217   80  723  487  141
  298   34  287  136   29  153  168   39  166  845   45  321  330  264
   94  164  363  265   44  246  150  556  463  195  226  274  186  449
  147  183  441  109  666  362  317  192  308  709  300  228  872  232
  241 

418

In [ ]:
# Loading 20 news group dataset
categories=['rec.sport.hockey','sci.electronics','talk.politics.guns','sci.space']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, remove=(['headers', 'footers', 'quotes']), shuffle=True, random_state=42)

'''
Cleaning: 
- filter out rare words, stopwords, and words that are not relevant to any of the 4 class labels.
- MI to select the top M ∈ [10, 100] feature words per class and take the union of all top feature words to train your multiclass model.

'''

download('punkt') #tokenizer, run once
download('stopwords') #stopwords dictionary, run once

# Tokenize, remove stopwords
stop_words = stopwords.words('english')

def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    # Restricts string to alphabetic characters only
    doc = [word for word in doc if word.isalpha()] 
    return doc

# text and ground truth labels
texts, y = twenty_train.data, twenty_train.target

corpus = [preprocess(text) for text in texts]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Task 1 : Data processing



## Task 1.2 : IMDB

 - X contain features based on the words in the movie reviews
 - y contain labels for whether the review sentiment is positive 1 or negative -1

In [ ]:
# IMDB train df


# QUESTION TO ANSWER: Do they make sense for calling a movie good and bad, respectively?


## Task 1.2 : Twenty Train

Text preprocessing, tokenizing and filtering of stopwords are all included in the CountVectorizer function


In [ ]:
# Turn the text content into feature vectors
count_vect = CountVectorizer() 
X_train_counts = count_vect.fit_transform(twenty_train.data) # builds a dict of features and transforms docs to feature vectors
X_train_counts.shape

# Occurences to frequences : TF_IDF

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
X = twenty_train.data # Test content

# Turn the text content into feature vectors
count_vect = CountVectorizer() 
X_train_counts = count_vect.fit_transform(twenty_train.data) # builds a dict of features and transforms docs to feature vectors
X_train_counts.shape

# Occurences to frequences : TF_IDF

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X = X_train_tfidf
X.todense()
twenty_train_arr = X.toarray()

y_cat = twenty_train.target
# convert array of class indices to one-hot encoded array
y = np.zeros((y_cat.size, y_cat.max() + 1))
y[np.arange(y_cat.size), y_cat] = 1

#print(zscore(twenty_train_arr))

# Task 2 : Implementing Classes

## Task 2.1 : Logistic Regression Class

In [111]:
# Logistic function
z = np.linspace(-5,5,100)
logistic = lambda z: 1./ (1 + np.exp(-z))

# From class Logistic Regression collab
class LogisticRegression:
    
    def __init__(self, add_bias=True, learning_rate=.1, epsilon=1e-4, max_iters=1e5, verbose=False):
        self.add_bias = add_bias
        self.learning_rate = learning_rate
        self.epsilon = epsilon                        #to get the tolerance for the norm of gradients 
        self.max_iters = max_iters                    #maximum number of iteration of gradient descent
        self.verbose = verbose

    def gradient(self, x, y):
        N,D = x.shape
        yh = logistic(np.dot(x, self.w))    # predictions  size N
        grad = np.dot(x.T, yh - y)/N        # divide by N because cost is mean over N points
        return grad
        
    def fit(self, x, y):
        if x.ndim == 1:
            x = x[:, None]
        if self.add_bias:
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])
        N,D = x.shape
        self.w = np.zeros(D)
        g = np.inf 
        t = 0
        # the code snippet below is for gradient descent
        while np.linalg.norm(g) > self.epsilon and t < self.max_iters:
            g = self.gradient(x, y)
            self.w = self.w - self.learning_rate * g 
            t += 1
        
        if self.verbose:
            print(f'terminated after {t} iterations, with norm of the gradient equal to {np.linalg.norm(g)}')
            print(f'the weight found: {self.w}')
        return self
    
    def predict(self, x):
        if x.ndim == 1:
            x = x[:, None]
        Nt = x.shape[0]
        if self.add_bias:
            x = np.column_stack([x,np.ones(Nt)])
        yh = logistic(np.dot(x,self.w))            #predict output
        return yh

# LogisticRegression.gradient = gradient  

In [ ]:
# Applying Logistic Regression on the IMDB data

model = LogisticRegression(verbose=True, add_bias=False)
fit = model.fit(IMDB_X_train,IMDB_y_train).predict(IMDB_X_train)

# plt.plot(X_train, y_train, '.', label='dataset')
# plt.plot(X_train, prediction, 'g', alpha=.5, label='predictions')
# plt.xlabel('x')
# plt.ylabel(r'$y$')
# plt.legend()
# # plt.show()
# plt.savefig('IMDB_prediction.png', bbox_inches="tight", dpi=300)

In [ ]:
'''
Check Gradient using small perturbation, 
monitor the cross-entropy as a function of iteration, 
and report your finding on both datasets
'''
# Initialize the gradient method of the LogisticRegression class with gradient function
# LogisticRegression.gradient = gradient             

In [ ]:
# Extra - Applying KNN on IMDB
knn = KNeighborsClassifier()
knn.fit(IMDB_X_train[:,:89523], IMDB_y_train)

pred = knn.predict(IMDB_X_test)[0]

print("Test Prediction for 0 : ", pred)

Test Prediction for 0 :  4.0


## Task 2.2 : Multiclass Regression Class

In [ ]:
# From class Multiclass Regression collab
class Multinomial_logistic:
    def __init__(self, nFeatures, nClasses):
        self.W = np.random.rand(nFeatures, nClasses)

    def predict(self, X):
        y_pred = np.exp(np.matmul(X, self.W))
        return y_pred / y_pred.sum(axis=1).reshape(X.shape[0], 1)

    def grad(self, X, y):
        return np.matmul(X.transpose(), self.predict(X) - y)

    def ce(self, X, y):
        return -np.sum(y * np.log(self.predict(X)))

    # modify it to add stopping criteria (what can you think of?)
    def fit(self, X, y, X_valid=None, y_valid=None, lr=0.005, niter=100):
        losses_train = np.zeros(niter)
        losses_valid = np.zeros(niter)
        for i in range(niter):
            self.W = self.W - lr * self.grad(X, y)
            loss_train = self.ce(X, y)
            losses_train[i] = loss_train
            if X_valid is not None and y_valid is not None:
                loss_valid = self.ce(X_valid, y_valid)
                losses_valid[i] = loss_valid
                print(f"iter {i}: {loss_train:.3f}; {loss_valid:.3f}")
            else:
                print(f"iter {i}: {loss_train:.3f}")
        return losses_train, losses_valid

    def check_grad(self, X, y):
        N, C = y.shape
        D = X.shape[1]

        diff = np.zeros((D, C))

        W = self.W.copy()

        for i in range(D):
            for j in range(C):
                epsilon = np.zeros((D, C))
                epsilon[i, j] = np.random.rand() * 1e-4

                self.W = self.W + epsilon
                J1 = self.ce(X, y)
                self.W = W

                self.W = self.W - epsilon
                J2 = self.ce(X, y)
                self.W = W

                numeric_grad = (J1 - J2) / (2 * epsilon[i, j])
                derived_grad = self.grad(X, y)[i, j]

                diff[i, j] = np.square(derived_grad - numeric_grad).sum() / \
                             np.square(derived_grad + numeric_grad).sum()

        # print(diff)
        return diff.sum()

    def evaluate(self, y, y_pred):
      accuracy = sum(y_pred.argmax(axis=1) == y.argmax(axis=1))
      accuracy = accuracy / y.shape[0]

      return accuracy

In [ ]:
# Applying Multiclass Regression on the 4-class prediction from the 20-news-group data

# Split the data into training, validation, and testing
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size = 0.33, random_state=1, shuffle=True)

X_train, X_valid, y_train, y_valid = model_selection.train_test_split(
    X_train, y_train, test_size = 0.5, random_state=1, shuffle=True)

#Create model object
D = X.shape[1]
C = y.shape[1]

mlr = Multinomial_logistic(D, C)

# check grad
print(mlr.check_grad(X_train, y_train))



## Task 2.3 : Method comparison using ROC curve

In [ ]:
models = [KNeighborsClassifier(),
          DecisionTreeClassifier(),
          sk_LogisticRegression()]

perf = {}

# our implementation is slow you may try sklearn version
logitreg = LogisticRegression(max_iters=1e3)
fit = logitreg.fit(X_train, y_train)
y_test_prob = fit.predict(X_test)
fpr, tpr, _ = roc_curve(y_test, y_test_prob)
auroc = roc_auc_score(y_test, y_test_prob)
perf["LogisticRegression (ours)"] = {'fpr':fpr, 'tpr':tpr, 'auroc':auroc}

for model in models:
    fit = model.fit(X_train, y_train)
    y_test_prob = fit.predict_proba(X_test)[:,1]
    fpr, tpr, _ = roc_curve(y_test, y_test_prob)
    auroc = roc_auc_score(y_test, y_test_prob)
    if type(model).__name__ == "LogisticRegression":
        perf["LogisticRegression (sklearn)"] = {'fpr':fpr,'tpr':tpr,'auroc':auroc}
    else:
        perf[type(model).__name__] = {'fpr':fpr,'tpr':tpr,'auroc':auroc}


plt.clf()
i = 0
for model_name, model_perf in perf.items():
    plt.plot(model_perf['fpr'], model_perf['tpr'],label=model_name)
    plt.text(0.4, i+0.1, model_name + ': AUC = '+ str(round(model_perf['auroc'],2)))
    i += 0.1

plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title('ROC in predicting Titanic survivor')
plt.legend(bbox_to_anchor=(1.04, 0.5), loc="upper left")
# plt.show()
plt.savefig("roc_curve.png", bbox_inches='tight', dpi=300)
# plt.close()

# Task 3 : Running Experiments

## Task 3.1 : Z-Scores
Report top 10 features with most positive/negative z-scores on the IMDB data using simple linear regression on the movie rating scores.

## Task 3.2.1: ROC and AUROC for Binary Classification

## Task 3.2.2 : Classification Accuracy for Multi-class Prediction